In [585]:
import pandas as pd

In [789]:
filepath = 'C:/Users/trevor.krause/Documents/Projects/Interceptions/'

In [592]:
df = pd.read_csv(filepath + '2019data.csv', low_memory=False)

In [593]:
players = pd.read_csv(filepath + 'players.csv')

In [594]:
df.columns.tolist()

['Unnamed: 0',
 'Date',
 'GameID',
 'play_id',
 'Drive',
 'qtr',
 'down',
 'time',
 'TimeUnder',
 'TimeSecs',
 'PlayTimeDiff',
 'SideofField',
 'yrdln',
 'yrdline100',
 'ydstogo',
 'ydsnet',
 'GoalToGo',
 'FirstDown',
 'posteam',
 'DefensiveTeam',
 'desc',
 'PlayAttempted',
 'Yards.Gained',
 'sp',
 'Touchdown',
 'ExPointResult',
 'TwoPointConv',
 'DefTwoPoint',
 'Safety',
 'Onsidekick',
 'PuntResult',
 'PlayType',
 'Passer',
 'Passer_ID',
 'PassAttempt',
 'PassOutcome',
 'PassLength',
 'AirYards',
 'YardsAfterCatch',
 'QBHit',
 'PassLocation',
 'InterceptionThrown',
 'Interceptor',
 'Rusher',
 'Rusher_ID',
 'RushAttempt',
 'RunLocation',
 'RunGap',
 'Receiver',
 'Receiver_ID',
 'Reception',
 'ReturnResult',
 'Returner',
 'BlockingPlayer',
 'Tackler1',
 'Tackler2',
 'FieldGoalResult',
 'FieldGoalDistance',
 'Fumble',
 'RecFumbTeam',
 'RecFumbPlayer',
 'Sack',
 'Challenge.Replay',
 'ChalReplayResult',
 'Accepted.Penalty',
 'PenalizedTeam',
 'PenaltyType',
 'PenalizedPlayer',
 'Penalty.Ya

In [595]:
# the PBP data addresses players by first initial and last name
# so we need to check if there are any players with similar names that would cause issues
dups = players.groupby(['name', 'Team'], as_index=False).agg({'Player': 'count'})

In [598]:
# These are players with the same name abbreviations that are on the same team
dups[dups['Player'] > 1]

,name,Team,Player
145,C.Covington,DAL,2
146,C.Davis,JAX,2
162,C.Jones,ARI,2
832,S.Griffin,SEA,2
940,T.Williams,GB,2


In [599]:
# T. Williams is the only of these to record an INT
# these were both Tramon, not Tim
df[df['Interceptor'].isin(dups[dups['Player'] > 1]['name'].tolist())][['Interceptor', 'DefensiveTeam']]

,Interceptor,DefensiveTeam
19498,C.Davis,TB
19499,C.Davis,TB
25884,T.Williams,GB
29863,C.Davis,TB
33103,T.Williams,GB


In [600]:
# so we get rid of Tim
players = players[players['Player'] != 'Tim Williams']

##### Defining an int

In [601]:
# limit to plays where an INT was thrown
ints = df[df['InterceptionThrown'] == 1]

In [602]:
len(ints[(ints['ChalReplayResult'] != 'Reversed')])

439

In [603]:
len(ints[(ints['ChalReplayResult'] == 'Reversed')])

21

In [604]:
# Of plays where ChalReplayResult == 'Reversed':
# 'No Play' at end of desc and/or official review reversing the INT means the play was not an INT. 
# Everything else is an INT 

# These are the 4 INTs
len(ints[(ints['ChalReplayResult'] == 'Reversed') & 
         ~((ints['desc'].str.contains('interception ruling, and the play was REVERSED')) | 
           (ints['desc'].str[-8:].str.contains('No Play.')))])

4

In [605]:
# create a condition to represent this
rev_stand = ((ints['ChalReplayResult'] == 'Reversed') & \
            ~((ints['desc'].str.contains('interception ruling, and the play was REVERSED')) | \
              (ints['desc'].str[-8:].str.contains('No Play.'))))

In [606]:
# of the 439 remaining with no reversal, the only thing that would nullify the INT is a penalty killing the play
len(ints[(ints['ChalReplayResult'] != 'Reversed')])

439

In [607]:
# this is represented by "No Play" in the desc
np = ((ints['ChalReplayResult'] != 'Reversed') & ~(ints['desc'].str.contains('No Play')))

In [608]:
# combining these conditions together gives us 410, which is the proper number according to profootballref
len(ints[rev_stand | np])

410

In [609]:
ints_full = ints[rev_stand | np]

In [626]:
# add interceptor full names and positions to the interception data
full = ints_full.merge(players[['Team', 'Pos', 'name', 'Player']].\
                       rename(columns={'Team':'DefensiveTeam', 'name': 'Interceptor'}), 
                       on=['DefensiveTeam', 'Interceptor'], how='left')

#### fix positions

In [627]:
full['Pos'].value_counts()

CB     146
FS      67
SS      44
DB      35
OLB     30
LB      21
MLB     19
ILB     11
DE      10
DT       4
SAF      3
NT       1
Name: Pos, dtype: int64

In [628]:
# create empty column where simplified position will be stored
full.loc[:, 'pos_abb'] = 'NA'

In [629]:
# defensive backs
full.loc[(full.loc[:, 'Pos'] == 'CB') | 
         (full.loc[:, 'Pos'] == 'FS') | 
         (full.loc[:, 'Pos'] == 'SS') | 
         (full.loc[:, 'Pos'] == 'DB') |
         (full.loc[:, 'Pos'] == 'SAF'), 'pos_abb'] = 'DB'

In [630]:
# linebackers
full.loc[(full.loc[:, 'Pos'] == 'OLB') | 
         (full.loc[:, 'Pos'] == 'ILB') | 
         (full.loc[:, 'Pos'] == 'LB') | 
         (full.loc[:, 'Pos'] == 'MLB'), 'pos_abb'] = 'LB'

In [631]:
# defensive lineman
full.loc[(full.loc[:, 'Pos'] == 'DE') | 
         (full.loc[:, 'Pos'] == 'DT') | 
         (full.loc[:, 'Pos'] == 'NT'), 'pos_abb'] = 'DL'

In [633]:
# players that dont have positon listed
full[full['pos_abb'] == 'NA'][['Interceptor', 'DefensiveTeam']]

,Interceptor,DefensiveTeam
14,V.Hargreaves,TB
75,Ja.Jenkins,NYG
76,Ja.Jenkins,NYG
80,M.Canady,BAL
82,D.Lawrence,CLE
91,M.Peters,LA
106,S.Bunting,TB
109,G.Conley,OAK
122,Ja.Jenkins,NYG
132,A.Sendejo,PHI


In [640]:
# We check profootballref and see that all of those players are DBs except for D Lawrence, who is a DL
full.loc[full['Interceptor'] == 'D.Lawrence', 'pos_abb'] = 'DL'

In [641]:
# after we fix him we can set the rest to DB
full.loc[full['pos_abb'] == 'NA', 'pos_abb'] = 'DB'

In [642]:
# players that dont have positon listed - empty now
full[full['pos_abb'] == 'NA']['Interceptor']

Series([], Name: Interceptor, dtype: object)

In [649]:
full.loc[full['Pos'].isna(), 'Pos'] = full['pos_abb'].copy()

##### Analysis

In [650]:
full['Pos'].value_counts()

CB     146
FS      67
DB      53
SS      44
OLB     30
LB      21
MLB     19
ILB     11
DE      10
DT       4
SAF      3
DL       1
NT       1
Name: Pos, dtype: int64

In [651]:
full['pos_abb'].value_counts()

DB    313
LB     81
DL     16
Name: pos_abb, dtype: int64

In [644]:
full['Pos'].value_counts()

CB     146
FS      67
SS      44
DB      35
OLB     30
LB      21
MLB     19
ILB     11
DE      10
DT       4
SAF      3
NT       1
Name: Pos, dtype: int64

In [652]:
# creat dummy variables from the categorical position variables
full[['DB', 'DL', 'LB']] = full['pos_abb'].str.get_dummies()

In [679]:
# group by passer
agg = \
full.groupby('Passer', as_index=False).\
    agg({'InterceptionThrown': 'count',
         'DB': 'sum',
         'DL': 'sum', 
         'LB': 'sum'
        })

In [680]:
# calculate percent from all INTs
agg['DBper'] = agg['DB']/agg['InterceptionThrown']
agg['LBper'] = agg['LB']/agg['InterceptionThrown']
agg['DLper'] = agg['DL']/agg['InterceptionThrown']

In [681]:
agg[agg['InterceptionThrown'] > 6]

,Passer,InterceptionThrown,DB,DL,LB,DBper,LBper,DLper
0,A.Dalton,14,10,1,3,0.714286,0.214286,0.071429
5,B.Mayfield,21,18,1,2,0.857143,0.095238,0.047619
11,C.Wentz,7,7,0,0,1.000000,0.000000,0.000000
14,D.Carr,8,8,0,0,1.000000,0.000000,0.000000
15,D.Haskins,7,6,0,1,0.857143,0.142857,0.000000
16,D.Hodges,8,7,0,1,0.875000,0.125000,0.000000
18,D.Jones,12,9,1,2,0.750000,0.166667,0.083333
20,D.Prescott,11,10,1,0,0.909091,0.000000,0.090909
21,D.Watson,12,9,0,3,0.750000,0.250000,0.000000
24,J.Allen,9,7,0,2,0.777778,0.222222,0.000000


##### Plot

In [666]:
import matplotlib.pyplot as plt

In [682]:
agg = agg[agg['InterceptionThrown'] > 6]

In [787]:
fig = plt.figure(figsize=(17, 6))

# plot # of Ints
agg = agg.sort_values('InterceptionThrown', ascending=False)

plt.subplot2grid((1, 2), (0, 0))
plt.bar(x=agg['Passer'], height=agg['DB'])
plt.bar(x=agg['Passer'], height=agg['LB'], bottom=agg['DB'])
plt.bar(x=agg['Passer'], height=agg['DL'], bottom=(agg['DB'] + agg['LB']))
plt.ylabel('# of Ints')
plt.xticks(rotation=65, fontsize=9)
plt.legend(['DB', 'LB', 'DL'])

# plot % of Ints
agg = agg.sort_values('DBper', ascending=True)

plt.subplot2grid((1, 2), (0, 1),)
plt.bar(x=agg['Passer'], height=agg['DBper'])
plt.bar(x=agg['Passer'], height=agg['LBper'], bottom=agg['DBper'])
plt.bar(x=agg['Passer'], height=agg['DLper'], bottom=(agg['DBper'] + agg['LBper']))
plt.ylabel('% of Ints')
plt.xticks(rotation=65, fontsize=9)
plt.legend(['DB', 'LB', 'DL'])

fig.suptitle('Passing Interceptions by Position of Interceptor', fontsize=15)
fig.subplots_adjust(wspace=.12, bottom=.17)

fig.savefig(filepath + 'plot.png')
plt.close()